Load data, convert strings to class labels
note for loading data: uncomment tfidf data codes and comment out count data codes to switch between data sets

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix

# train_data = open("train_tfidf.csv",'r').readlines()
# dev_data = open("dev_tfidf.csv",'r').readlines()
# test_data = open("test_tfidf.csv",'r').readlines()
# vocab_data = open("vocab.txt",'r').readlines()

train_data = open("train_count.csv",'r').readlines()
dev_data = open("dev_count.csv",'r').readlines()
test_data = open("test_count.csv",'r').readlines()
vocab_data = open("vocab.txt",'r').readlines()

x_train = []
y_train = []
x_dev = []
y_dev = []
x_test = []
y_test = []
tuple_list = []
tfidf_list = []
label_id = []
tuple_id = []
tuple_tfidf = []
dev_tuple_id = []
dev_tuple_tfidf = []
dev_tuple_list = []
vocab_list = []

target = {91:None, 93:None, 34:None, 40:None, 41:None}
def convert_feature(raw):
    if raw == "neg": return 2
    elif raw == "neu": return 1
    elif raw == "pos": return 0

for instance in vocab_data:
    instance = instance.strip().split('\t')
    vocab_list.append(instance)

#train data
for instance in train_data[1:]:
    instance = str(instance).translate(target)
    instance = instance.strip().split(',')
    tuple_id = list(instance[2::2])
    tuple_tfidf = list(instance[3::2])
    tuple_list.append(list(zip(tuple_id, tuple_tfidf)))
    y_train.append(convert_feature(instance[0]))
# tuple_array = np.array(tuple_list, dtype = object)
# y_train = np.array(y_train, dtype = float)

#dev data
for instance in dev_data[1:]:
    instance = str(instance).translate(target)
    instance = instance.strip().split(',')
    dev_tuple_id = list(instance[2::2])
    dev_tuple_tfidf = list(instance[3::2])
    dev_tuple_list.append(list(zip(dev_tuple_id, dev_tuple_tfidf)))
    y_dev.append(convert_feature(instance[0]))
# x_dev = np.array(x_dev, dtype = float)
# y_dev = np.array(y_dev, dtype = float)

#train data matrices for sparse matrix
r_count=0
i=[]
j=[]
k=[]
for r in tuple_list:
    for c in r:
        i.append(r_count)
        j.append(int(c[0]))
        k.append(float(c[1]))
    r_count += 1

#dev data matrices for sparse matrix
r_count_dev=0
i_dev=[]
j_dev=[]
k_dev=[]
for r in dev_tuple_list:
    for c in r:
        i_dev.append(r_count_dev)
        j_dev.append(int(c[0]))
        k_dev.append(float(c[1]))
    r_count_dev += 1

# make sparse matrices
x_train = coo_matrix((k, (i,j)))
x_dev = coo_matrix((k_dev, (i_dev,j_dev)))

Baseline

In [3]:
from sklearn.dummy import DummyClassifier
dclf = DummyClassifier(strategy = "most_frequent")
dclf.fit(x_train,y_train)
dclf_predictions = dclf.predict(x_dev)
dclf.score(x_dev,y_dev)

0.406661308148297

Build and train classifier using train data, then output predictions of x_dev

In [4]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

mnb = MultinomialNB()
mnb.fit(x_train,y_train)
mnb_predictions = mnb.predict(x_dev)

lr = LogisticRegression()
lr.fit(x_train,y_train)
lr_predictions = lr.predict(x_dev)

sclf = LinearSVC()
sclf.fit(x_train,y_train)
sclf_predictions = sclf.predict(x_dev)

# dt = DecisionTreeClassifier(random_state = 0)
# dt.fit(x_train,y_train)
# dt_predictions = dt.predict(x_dev)

rf = RandomForestClassifier(max_depth = 60, random_state = 0)
rf.fit(x_train,y_train)
rf_predictions = rf.predict(x_dev)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\User\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Evaluation metrics

In [5]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

dclf_acc = accuracy_score(y_dev, dclf_predictions)
dclf_f1 = f1_score(y_dev, dclf_predictions, average = 'macro')

mnb_acc = accuracy_score(y_dev, mnb_predictions)
mnb_f1 = f1_score(y_dev, mnb_predictions, average = 'macro')

sclf_acc = accuracy_score(y_dev, sclf_predictions)
sclf_f1 = f1_score(y_dev, sclf_predictions, average = 'macro')

# dt_acc = accuracy_score(y_dev, dt_predictions)
# dt_f1 = f1_score(y_dev, dt_predictions, average = 'macro')

rf_acc = accuracy_score(y_dev, rf_predictions)
rf_f1 = f1_score(y_dev, rf_predictions, average = 'macro')

lr_acc = accuracy_score(y_dev, lr_predictions)
lr_f1 = f1_score(y_dev, lr_predictions, average = 'macro')

print("dclf_acc = ", dclf_acc)
print("dclf_f1 = ", dclf_f1)
print("mnb_acc = ", mnb_acc)
print("mnb_f1 = ", mnb_f1)
print("sclf_acc = ", sclf_acc)
print("sclf_f1 = ", sclf_f1)
# print("dt_acc = ", dt_acc)
# print("dt_f1 = ", dt_f1)
print("rf_acc = ", rf_acc)
print("rf_f1 = ", rf_f1)
print("lr_acc = ", lr_acc)
print("lr_f1 = ", lr_f1)

dclf_acc =  0.406661308148297
dclf_f1 =  0.19273121198052448
mnb_acc =  0.7307846880337586
mnb_f1 =  0.7466609947326751
sclf_acc =  0.7408319099768914
sclf_f1 =  0.7564541742479624
rf_acc =  0.6364412739877424
rf_f1 =  0.6223057385849048
lr_acc =  0.7406812016477444
lr_f1 =  0.7563062718161794


Predictions for Kaggle using prefered classifier (lr)

In [7]:
test_tuple_id = []
test_tuple_tfidf = []
test_tuple_list = []
test_tweet_id = []

#load test tfidf matrix
for instance in test_data[1:]:
    instance = str(instance).translate(target)
    instance = instance.strip().split(',')
    test_tuple_id = list(instance[2::2])
    test_tuple_tfidf = list(instance[3::2])
    test_tuple_list.append(list(zip(test_tuple_id, test_tuple_tfidf)))
    test_tweet_id.append(instance[1])
    
#test data matrices for sparse matrix
r_count_test=0
i_test=[]
j_test=[]
k_test=[]
for r in test_tuple_list:
    for c in r:
        i_test.append(r_count_test)
        j_test.append(int(c[0]))
        k_test.append(float(c[1]))
    r_count_test += 1
x_test = coo_matrix((k_test, (i_test,j_test)))

clf = lr
test_predictions = clf.predict(x_test)

def convert_sentiment(raw):
    if raw == 2: return "neg"
    elif raw == 1: return "neu"
    elif raw == 0: return "pos"

with open('predictions.csv','w+') as pf:
    pf.write('tweet_id,sentiment\n')
    for prediction, tweet_id in zip(test_predictions,test_tweet_id):
        pf.write(str(tweet_id) + ',')
        pf.write(convert_sentiment(prediction) + '\n')